<h1>第9章: ベクトル空間法</h1>

enwiki-20150112-400-r10-105752.txt.bz2は，2015年1月12日時点の英語のWikipedia記事のうち，約400語以上で構成される記事の中から，ランダムに1/10サンプリングした105,752記事のテキストをbzip2形式で圧縮したものである．<br>
このテキストをコーパスとして，単語の意味を表すベクトル（分散表現）を学習したい．<br>
第9章の前半では，コーパスから作成した単語文脈共起行列に主成分分析を適用し，単語ベクトルを学習する過程を，いくつかの処理に分けて実装する．<br>
第9章の後半では，学習で得られた単語ベクトル（300次元）を用い，単語の類似度計算やアナロジー（類推）を行う．<br>

なお，問題83を素直に実装すると，大量（約7GB）の主記憶が必要になる． <br>
メモリが不足する場合は，処理を工夫するか，1/100サンプリングのコーパスenwiki-20150112-400-r100-10576.txt.bz2を用いよ．

<h2>目次</h2>

- [80. コーパスの整形](#prob80)
- [81. 複合語からなる国名への対処](#prob81)
- [82. 文脈の抽出](#prob82)
- [83. 単語/文脈の頻度の計測](#prob83)
- [84. 単語文脈行列の作成](#prob84)
- [85. 主成分分析による次元圧縮](#prob85)
- [86. 単語ベクトルの表示](#prob86)
- [87. 単語の類似度](#prob87)
- [88. 類似度の高い単語10件](#prob88)
- [89. 加法構成性によるアナロジー](#prob89)

In [1]:
!mkdir data

In [2]:
!mkdir work

**とりあえずコンパクトにやりたいから1/100コーパス使う**

In [13]:
!wget -NP ./data/ http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r100-10576.txt.bz2

--2018-08-22 14:46:48--  http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r100-10576.txt.bz2
www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp) をDNSに問いあわせています... 130.34.192.83
www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)|130.34.192.83|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 22022468 (21M) [application/x-bzip2]
`./data/enwiki-20150112-400-r100-10576.txt.bz2' に保存中

enwiki-20150112-400 100%[===================>]  21.00M  3.52MB/s 時間 5.0s     

2018-08-22 14:46:54 (4.23 MB/s) - `./data/enwiki-20150112-400-r100-10576.txt.bz2' へ保存完了 [22022468/22022468]



In [14]:
!bunzip2 --help

bzip2, a block-sorting file compressor.  Version 1.0.6, 6-Sept-2010.

   usage: bunzip2 [flags and input files in any order]

   -h --help           print this message
   -d --decompress     force decompression
   -z --compress       force compression
   -k --keep           keep (don't delete) input files
   -f --force          overwrite existing output files
   -t --test           test compressed file integrity
   -c --stdout         output to standard out
   -q --quiet          suppress noncritical error messages
   -v --verbose        be verbose (a 2nd -v gives more)
   -L --license        display software version & license
   -V --version        display software version & license
   -s --small          use less memory (at most 2500k)
   -1 .. -9            set block size to 100k .. 900k
   --fast              alias for -1
   --best              alias for -9

   If invoked as `bzip2', default action is to compress.
              as `bunzip2',  default action is to decompress.
      

In [15]:
!bunzip2 -k ./data/enwiki-20150112-400-r100-10576.txt.bz2

In [16]:
!cat ./data/enwiki-20150112-400-r100-10576.txt 2>/dev/null | head

Anarchism

Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions, but that several authors have defined as more specific institutions based on non-hierarchical free associations. Anarchism holds the state to be undesirable, unnecessary, or harmful. While anti-statism is central, anarchism entails opposing authority or hierarchical organisation in the conduct of human relations, including, but not limited to, the state system.
As a subtle and anti-dogmatic philosophy, anarchism draws on many currents of thought and strategy. Anarchism does not offer a fixed body of doctrine from a single particular world view, instead fluxing and flowing as a philosophy. There are many types and traditions of anarchism, not all of which are mutually exclusive. Anarchist schools of thought can differ fundamentally, supporting anything from extreme individualism to complete collectivism. Strains of anarchism have often been divided int

<a name='prob80'></a>
# 80. コーパスの整形

文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． <br>
ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． <br>
そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．<br>

トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"<br>
空文字列となったトークンは削除<br>
以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [26]:
with open('./data/enwiki-20150112-400-r100-10576.txt') as f, open('./work/text_tokenized.txt', 'w') as out:
    for line in f:
        tokens = line.rstrip().split()
        print(*(token.strip('.,!?;:()[]\'"') for token in tokens if len(token.strip('.,!?;:()[]\'"')) > 0), file=out)

token.rstrip...って2回書きたくないけど1トークンごとに書き込みたくなかった + もう1つ配列用意してappendするのも効率悪そう

<a name='prob81'></a>
# 81. 複合語からなる国名への対処

英語では，複数の語の連接が意味を成すことがある．<br>
例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．<br>
そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．<br>
しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．<br>

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．<br>
例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

<h4>pandasがHTMLの\<table>タグのスクレイピングにクッソ便利な件</h4>

In [4]:
import pandas as pd
df = pd.io.html.read_html('https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_and_their_capitals_in_native_languages',
                         skiprows=[0])

In [5]:
df[0]

,0,1,2,3,4
0,Afghanistan,Kabul,Afghanestanافغانستان,Kabulكابل,Pashto/Dari(Arabic script)
1,Albania,Tirana,Shqipëria,Tirana,Albanian
2,Algeria,Algiers,DzayerⴷⵣⴰⵢⴻⵔAl-Jazā'irالجزائر,DzayerⴷⵣⴰⵢⴻⵔAl-Jazā'irالجزائر,Berber language(Tifinagh script)Arabic(Arabic ...
3,American Samoa[1],Pago Pago,Amerika SāmoaAmerican Samoa,Pago Pago Pago Pago,SamoanEnglish
4,Andorra,Andorra la Vella,Andorra,Andorra la Vella,Catalan
5,Angola,Luanda,Angola Ngola,Luanda Lwanda,Portuguese Kongo
6,Anguilla[2],The Valley,Anguilla,The Valley,English
7,Antigua and Barbuda,Saint John's,Antigua and Barbuda,St. John's,English
8,Argentina,Buenos Aires,Argentina,Buenos Aires,Spanish
9,Armenia,Yerevan,HayastánՀայաստան,YerevanԵրեվան,Armenian(Armenian alphabet)


- Xから始まる国はないという罠 (最初range(26)で回してた)

In [6]:
import re

In [7]:
countries = [re.sub('\[[0-9]+\]', '', df[table_num][0][row]) for table_num in range(25) for row in range(len(df[table_num]))]

In [8]:
countries[:10]

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia']

()がじゃまなやつとかあったからそれも削除

In [9]:
regex_link = re.compile('\[[0-9]+\]')
regex_paren = re.compile('\s\(.+\)')

In [10]:
countries = [regex_paren.sub('', regex_link.sub('', df[table_num][0][row])) for table_num in range(25) for row in range(len(df[table_num]))]

In [11]:
countries[:10]

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia']

In [12]:
countries_comp = set(country for country in countries if ' ' in country)

In [13]:
countries_comp

{'American Samoa',
 'Antigua and Barbuda',
 'Bosnia and Herzegovina',
 'British Virgin Islands',
 'Burkina Faso',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Christmas Island',
 'Cocos Islands',
 'Cook Islands',
 'Costa Rica',
 'Czech Republic',
 "Côte d'Ivoire(formerly Ivory Coast)",
 'Democratic Republic of the Congo',
 'Dominican Republic',
 'East Timor',
 'El Salvador',
 'Equatorial Guinea',
 'Eswatini(formerly Swaziland)',
 'Falkland Islands',
 'Faroe Islands',
 'Federated States of Micronesia',
 'French Guiana',
 'French Polynesia',
 'Hong Kong',
 'Isle of Man',
 'Marshall Islands',
 'New Caledonia',
 'New Zealand',
 'Norfolk Island',
 'North Korea',
 'Northern Mariana Islands',
 'Papua New Guinea',
 'Pitcairn Islands',
 'Puerto Rico',
 'Republic of the Congo',
 'Sahrawi Arab Democratic Republic',
 'Saint Barthélemy',
 'Saint Helena, Ascension and Tristan da Cunha',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint Martin',
 'Saint Pierre and Miquelon',
 'Sai

In [16]:
with open('work/text_tokenized.txt') as f, open('./work/text_tokenized_comp.txt', 'w') as out:
    for line in f:
        for country in sorted(countries_comp, key=lambda x: len(x), reverse=True):
            line = line.replace(country, '_'.join(country.split()))
        print(line.rstrip(), file=out)

長い文から順に見ていくと国名の一部がかぶっている場合に最長一致を見てこられる

<a name='prob82'></a>
# 82. 文脈の抽出

81で作成したコーパス中に出現するすべての単語$t$に関して，単語$t$と文脈語$c$のペアをタブ区切り形式ですべて書き出せ．<br>
ただし，文脈語の定義は次の通りとする．<br>

- ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語$t$そのものは含まない）
- 単語$t$を選ぶ度に，文脈幅$d$は$\{1,2,3,4,5\}$の範囲でランダムに決める．

In [23]:
import random

In [25]:
with open('./work/text_tokenized_comp.txt') as f, open('./work/context.txt' ,'w') as out:
    for line in f:
        words = line.rstrip().split()
        for idx, word in enumerate(words):
            window = random.randint(1,5)
            context = [words[idx+i] for i in range(-window,window+1) if 0 <= idx+i < len(words) and i != 0]
            if len(context) > 0:
                for c in context:
                    print(word, c, sep='\t', file=out)

In [26]:
!head ./work/context.txt

Anarchism	is
Anarchism	a
Anarchism	political
Anarchism	philosophy
is	Anarchism
is	a
a	Anarchism
a	is
a	political
a	philosophy


In [27]:
!wc -l ./work/context.txt

68091560 ./work/context.txt


<a name='prob83'></a>
# 83. 単語／文脈の頻度の計測

82の出力を利用し，以下の出現分布，および定数を求めよ．<br>

$f(t,c)$: 単語$t$と文脈語$c$の共起回数<br>
$f(t,∗)$: 単語$t$の出現回数<br>
$f(∗,c)$: 文脈語$c$の出現回数<br>
$N$: 単語と文脈語のペアの総出現回数

**この実装は1行に注目単語と複数のcontextが書かれている出力についてのものになっている (tが複数回カウントされることを考える必要がない)**

(82でfor文使わないで`print(word, *context, sep='\t', file=out)`する)

In [1]:
from collections import Counter

In [36]:
f_tc = Counter()
f_t = Counter()
f_c = Counter()
N = 0

In [37]:
with open('./work/context.txt') as f:
    for line in f:
        word, contexts = line.rstrip().split('\t', maxsplit=1)
        f_t.update({word:1})
        for context in contexts.split('\t'):
            f_c.update({context:1})
            f_tc.update({(word, context):1})
            N += 1

In [38]:
f_t.most_common(10)

[('the', 754586),
 ('of', 410555),
 ('and', 357978),
 ('in', 283554),
 ('to', 262920),
 ('a', 234498),
 ('was', 137710),
 ('The', 119810),
 ('is', 106471),
 ('for', 98698)]

In [39]:
f_tc.most_common(10)

[(('the', 'of'), 282255),
 (('of', 'the'), 282089),
 (('the', 'the'), 157765),
 (('the', 'in'), 129301),
 (('in', 'the'), 129221),
 (('the', 'and'), 115802),
 (('and', 'the'), 115350),
 (('the', 'to'), 107403),
 (('to', 'the'), 106905),
 (('and', 'of'), 57127)]

ループ毎回回すの非効率だからpickleでシリアライズして保存しとく(**jsonでよくね?**)

In [15]:
import pickle

In [51]:
with open('./work/variables.pickle', 'wb') as out:
    pickle.dump({'f_tc':f_tc, 'f_t':f_t, 'f_c':f_c, 'N':N}, out)

tに当たる単語をソートして逐次f_t, f_tcを出力していけば良い

ファイルを分割してそれぞれtでソートした上でマージソート的にくっつけるなど

<a name='prob84'></a>
# 84. 単語文脈行列の作成

83の出力を利用し，単語文脈行列$X$を作成せよ．<br>
ただし，行列$X$の各要素$X_{tc}$は次のように定義する．<br>

- $f(t,c)≥10$ならば，$X_{tc}=PPMI(t,c)=\max\{\log \frac{N×f(t,c)}{f(t,∗)×f(∗,c)},0\}$
- $f(t,c)<10$ならば，$X_{tc}=0$<br>
ここで，$PPMI(t,c)$はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．<br>
なお，行列$X$の行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．<br>
幸い，行列$X$のほとんどの要素は$0$になるので，非$0$の要素だけを書き出せばよい．

In [16]:
with open('./work/variables.pickle', 'rb') as f:
    dic = pickle.load(f)
    f_tc = dic.get('f_tc')
    f_t = dic.get('f_t')
    f_c = dic.get('f_c')
    N = dic.get('N')

In [3]:
import numpy as np
from scipy import sparse, io

In [70]:
X = sparse.lil_matrix((len(f_t), len(f_c)))

In [71]:
X

<383369x383369 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in LInked List format>

In [72]:
import math

def calc_ppmi(t, c, f_tc:dict, f_t:dict, f_c:dict, N):
    num = N * f_tc.get((t,c))
    deno = f_t.get(t) * f_c.get(c)
    pmi = math.log2(num / deno)
    return max(pmi, 0)

tokenからindexへの辞書

In [17]:
word2idx_t = {w:i for i, (w, _) in enumerate(f_t.items())}
word2idx_c = {w:i for i, (w, _) in enumerate(f_c.items())}

後々必要になったので逆も

In [37]:
idx2word_t = {i:w for i, (w, _) in enumerate(f_t.items())}

In [74]:
for (t, c), count in f_tc.items():
    if count >= 10:
        idx_t = word2idx_t[t]
        idx_c = word2idx_c[c]
        X[idx_t, idx_c] = calc_ppmi(t, c, f_tc, f_t, f_c, N)

In [75]:
X

<383369x383369 sparse matrix of type '<class 'numpy.float64'>'
	with 632691 stored elements in LInked List format>

In [76]:
X.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [1.38586242, 3.3644911 , 1.54082702, ..., 0.        , 0.        ,
         0.        ],
        [3.39695674, 1.48369332, 3.00200609, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [80]:
io.savemat('./work/matX', {'X':X})

<a name='prob85'></a>
# 85. 主成分分析による次元圧縮

84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

In [4]:
X = io.loadmat('./work/matX.mat')['X']

In [5]:
from sklearn.decomposition import PCA

In [6]:
pca = PCA(n_components=300)
pca.fit(X)

TypeError: PCA does not support sparse input. See TruncatedSVD for a possible alternative.

PCAじゃなくてTruncatedSVD使えって

In [7]:
from sklearn.decomposition import TruncatedSVD

In [8]:
pca = TruncatedSVD(n_components=300)
pca.fit(X)

TruncatedSVD(algorithm='randomized', n_components=300, n_iter=5,
       random_state=None, tol=0.0)

<a name='prob86'></a>
# 86. 単語ベクトルの表示

85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．<br>
ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

In [12]:
X_embedded = pca.transform(X)

In [13]:
X_embedded.shape

(383369, 300)

In [22]:
idx_united_states = word2idx_t['United_States']
print(X_embedded[idx_united_states])

[ 2.45108459e+01 -3.96679861e+00 -9.18404761e+00  1.60167891e+00
  9.56103481e+00 -1.14245349e+00 -1.58367888e+01 -8.63907950e+00
 -1.51435603e+00  5.07648537e-01 -5.50641693e-01 -8.57936355e+00
  1.77031272e+00  9.74590868e-02  8.09741230e+00  2.13172530e+00
 -7.51825284e-01  8.57514615e-01  9.66519170e-02 -6.06572566e+00
  1.97906481e+00  6.58550744e+00 -1.68446192e+00  6.14908757e-01
 -5.30749590e+00 -8.67372735e-01  3.56654361e+00 -8.12828922e+00
 -2.03155094e+00  2.62649428e+00 -2.83809096e+00  5.83318270e+00
  5.64428642e-01  7.61583400e-01  8.68881950e+00 -1.14069945e+00
 -7.27356852e+00 -3.81588142e+00 -3.92720729e+00  1.59670589e+00
 -1.03880298e+00  4.50792676e-01 -5.75155190e+00  8.82337473e+00
  8.52334128e-01  3.90851358e+00  2.20687669e+00  5.85969696e+00
 -1.56461942e-01 -1.21757910e+00 -6.18491797e-02  7.74413894e-01
  4.62218911e+00  4.58044202e-01  6.17849536e-01  3.48293679e+00
  2.69087197e+00  2.32873355e+00  4.84194081e-01 -1.42597133e+00
 -1.15087354e+00 -7.15916

<a name='prob87'></a>
# 87. 単語の類似度

85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．<br>
ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
vec_united_states = np.array([X_embedded[idx_united_states]]).astype(np.float32)
idx_us = word2idx_t['U.S']
vec_us = np.array([X_embedded[idx_us]]).astype(np.float32)

結構よさそう。

In [81]:
cos_sim = cosine_similarity(vec_united_states, vec_us)[:,0]
print(cos_sim[0])

0.8260192


<a name='prob88'></a>
# 88. 類似度の高い単語10件

85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

In [75]:
idx_england = word2idx_t['England']
vec_england = np.array([X_embedded[idx_england]]).astype(np.float32)

- sliceについて
cos_simが2次元配列で返ってくる -> どうせ1列しかないから列のindex0を拾う -> argsortは昇順に返るから\[::-1\]で逆順に + England自体は含めたくないからスライスのインデックスは-2から

In [77]:
cos_sim = cosine_similarity(X_embedded, vec_england)[:,0]
indices_in_sim_order = np.argsort(cos_sim, axis=0)[-2::-1]

In [78]:
indices_in_sim_order

array([  5976,   4596,    268, ...,  47021, 216891, 369880])

動作確認

In [79]:
idx2word_t[idx_england]

'England'

In [80]:
import itertools
for idx in itertools.islice(indices_in_sim_order,10):
    print(idx2word_t[idx], cos_sim[idx])

Australia 0.6709476764615532
Wales 0.631520600202294
France 0.6306139123660406
Italy 0.6056774585487458
Japan 0.603044066132867
Germany 0.6018050945339366
Scotland 0.5980053601468223
Ireland 0.561732085974481
Europe 0.5611709314566591
India 0.5550795405307398


<a name='prob89'></a>
# 89. 加法構成性によるアナロジー

85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

In [82]:
idx_spain = word2idx_t['Spain']
vec_spain = np.array([X_embedded[idx_spain]]).astype(np.float32)
idx_madrid = word2idx_t['Madrid']
vec_madrid = np.array([X_embedded[idx_madrid]]).astype(np.float32)
idx_athens = word2idx_t['Athens']
vec_athens = np.array([X_embedded[idx_athens]]).astype(np.float32)

In [83]:
vec_comp = vec_spain - vec_madrid + vec_athens

In [85]:
cos_sim = cosine_similarity(X_embedded, vec_comp)[:,0]
indices_in_sim_order = np.argsort(cos_sim, axis=0)[::-1]

In [86]:
for idx in itertools.islice(indices_in_sim_order,10):
    print(idx2word_t[idx], cos_sim[idx])

Spain 0.9000651389763585
Sweden 0.7860695296781051
Italy 0.7786093381971849
Austria 0.7739187200547092
Netherlands 0.7362301590224218
Denmark 0.7137012568293781
Britain 0.7122905688391622
Germany 0.6991136996603655
Portugal 0.6862837483585569
Russia 0.6845829060472209


うーん...
<h4>ちなみにGreeceは?</h4>

In [88]:
idx_greece = word2idx_t['Greece']

In [91]:
print(list(indices_in_sim_order).index(idx_greece))
print(cos_sim[idx_greece])

14
0.6540359601222318
